# Semantic Kernel を使ったサンプルNotebook

このノートブックは英語版 '10-semantic-kernel.ipynb' と同等の構成で、旅行先とフライト時間取得のシンプルなエージェント例を示します。
関数呼び出し（Function Calling）のストリーミング挙動と、失敗時にバックアップ関数へフォールバックするパターンの観察を目的としています。

In [20]:
# ポイント: StreamingTextContent で逐次観測 / 型注釈でメタ明確化。
# 秘密は .env (load_dotenv) 管理しハードコード回避。Azure 時は AzureChatCompletion。
# 非 Azure か未設定時は GitHub Inference(OpenAIChatCompletion) にフォールバック。
import json
import os

from typing import Annotated
from dotenv import load_dotenv
from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function

In [21]:
# フォールバック設計意図
# - 一次 get_flight_times は常に固定エラー: 「失敗→判定→代替呼出」流れを学習用に強制し可視化します。
# - 目的: (1) 可用性を get_flight_times_backup で維持 (2) ハルシネーションの抑制 (3) 判定を単純化し 'HTTP ERROR 404' を返す。
# - 都市辞書は最小データ: ノイズを減らしフォールバック挙動理解を優先します。
# 実運用では例外種別 + リトライ/バックオフ/メトリクス/サーキットブレーカ等を組み合わせます。
class DestinationsPlugin:
    """休暇用の目的地一覧プラグイン。"""

    @kernel_function(description="休暇用候補地一覧を返します")
    def get_destinations(self) -> Annotated[str, "候補地一覧"]:
        return """
                Barcelona, Spain
                Paris, France
                Berlin, Germany
                Tokyo, Japan
                New York, USA
                """

    @kernel_function(description="指定都市のフライト時間を返します (一次サービス: 常にエラーを返す例)")
    def get_flight_times(self, destination: Annotated[str, "都市名"]) -> Annotated[str, "フライト時間"]:
        return 'HTTP ERROR 404: Flight times service is currently unavailable.'

    @kernel_function(description="バックアップ: 指定都市のフライト時間を返します")
    def get_flight_times_backup(self, destination: Annotated[str, "都市名"]) -> Annotated[str, "フライト時間"]:
        flight_times = {
            'Barcelona': ['08:30 AM', '02:15 PM', '10:45 PM'],
            'Paris': ['06:45 AM', '12:30 PM', '07:15 PM'],
            'Berlin': ['07:20 AM', '01:45 PM', '09:30 PM'],
            'Tokyo': ['11:00 AM', '05:30 PM', '11:55 PM'],
            'New York': ['05:15 AM', '03:00 PM', '08:45 PM']
        }
        city = destination.split(',')[0].strip()
        if city in flight_times:
            times = ', '.join(flight_times[city])
            return f'フライト時間 ({city}): {times}'
        return f'{city} の情報はありません。'

In [23]:
# モデル初期化戦略 (Fallback Priorities)
# 1) Azure OpenAI (AzureChatCompletion) : 低レイテンシ / Azure OpenAI エンドポイント API / api_version 指定
# 2) Azure AI Foundry Project (AzureAIInferenceChatCompletion) : Azure AI Foundry Project 用のエンドポイントで Azure OpenAI 以外の AI Service も参照対象
# 3) GitHub Inference (OpenAI 互換) : 学習用フォールバックでハンズオン停止を防止
# 目的: 可用性の維持、障害のポイントを可視化、早期に失敗を検知。
from dotenv import load_dotenv
load_dotenv()

# --- 環境変数取得 ---
azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')  # Azure OpenAI / Foundry 共通利用可
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')  # 例: https://<resource>.openai.azure.com/
azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-10-21')
azure_chat_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
project_endpoint = os.getenv('PROJECT_ENDPOINT')  # Azure AI Foundry Project Endpoint
model_deployment_name = os.getenv('MODEL_DEPLOYMENT_NAME', azure_chat_deployment)

# --- インポート (存在しないクラスは無視) ---
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
try:
    from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion  # type: ignore
except Exception:
    AzureAIInferenceChatCompletion = None  # noqa: N816

chat_completion_service = None
path = None

try:
    if azure_api_key and azure_endpoint:
        chat_completion_service = AzureChatCompletion(
            service_id="azure-openai",
            deployment_name=azure_chat_deployment,
            endpoint=azure_endpoint,
            api_key=azure_api_key,
            api_version=azure_api_version,
        )
        path = 'Azure OpenAI'
    elif project_endpoint and model_deployment_name and AzureAIInferenceChatCompletion:
        try:
            chat_completion_service = AzureAIInferenceChatCompletion(
                service_id="foundry",
                deployment_name=model_deployment_name,
                endpoint=project_endpoint,
                api_key=azure_api_key,
            )
        except TypeError as e:
            raise RuntimeError(f'AzureAIInferenceChatCompletion 初期化失敗: {e} (semantic-kernel バージョン差異を確認してください)')
        path = 'Azure AI Foundry Project'
    else:
        inference_token = os.getenv('GITHUB_TOKEN')
        if not inference_token:
            raise RuntimeError('AZURE_OPENAI_API_KEY か GITHUB_TOKEN を .env に設定してください。')
        client = AsyncOpenAI(
            api_key=inference_token,
            base_url='https://models.inference.ai.azure.com/',
        )
        chat_completion_service = OpenAIChatCompletion(
            ai_model_id='gpt-4o-mini',
            async_client=client,
        )
        path = 'GitHub Inference'
except Exception as e:
    raise RuntimeError(f'チャットサービス初期化全体で失敗しました: {e}')

if chat_completion_service is None:
    raise RuntimeError('chat_completion_service の初期化に失敗しました。')

print(f"# モデル初期化完了 (Endpoint: {path})")

# モデル初期化完了 (Endpoint: Azure OpenAI)


In [24]:
# このセルのポイント:
# - プロンプト内で「処理手順 / ガイドライン / 目的」を構造化し、暗黙知を減らして挙動の安定性を高めています。
# - 失敗判定条件 ('HTTP ERROR' を含む) をテキストで明示し、モデル判断の再現性を向上させています。
# - 都市名ホワイトリストを指示で固定し、ツール引数揺れによる失敗を防ぎます。
# - フォールバック利用時のユーザー向けメッセージ規約を定義し、エラー体験を標準化します。
# - 目的を明示し長い対話でも逸脱を防ぎます。
AGENT_NAME = 'TravelAgent'
AGENT_INSTRUCTIONS = """
あなたはフライト情報と旅行アクティビティ提案を行うフライト予約エージェントです。旅行アクティビティ提案はユーザー、場所、滞在時間に適合させます。

【利用可能ツール】
1. get_destinations: 利用可能な旅行先候補一覧を返します。
2. get_flight_times: 指定都市のフライト時間 (一次サービス。ここでは常にエラーを返す挙動) を返します。
3. get_flight_times_backup: 一次サービス障害時に利用するバックアップのフライト時間を返します。

【支援手順】
- ユーザーがフライト予約を依頼したら最初に get_flight_times を呼び出し最も早い便を「仮予約した」として提示します。
- get_flight_times の戻り値に 'HTTP ERROR' などのエラー文字列が含まれる場合は、同じ destination 引数で直ちに get_flight_times_backup を呼び出します。
- フォールバックを使用した場合は、回答本文の冒頭で「一次サービス障害のためバックアップデータを使用しています。」と前向きなトーンで明示します。
- 実際の予約システムは存在しないため、追加確認を求めず仮予約完了として説明します。
- 会話履歴から嗜好 (例: 早朝便志向 / 滞在時間重視 など) を推測し、提案時に根拠を一行で明示します。

【ガイドライン】
- ツール呼び出し時は都市名 (Barcelona, Paris, Berlin, Tokyo, New York) を正確に使用します。
- フライト時間は箇条書き (ハイフンまたは番号) で表示します。
- フォールバック利用時はポジティブで信頼感のあるトーンを保ちつつ代替データ利用を説明します。
- 提示する時間帯が非現実的 / 重複している場合は自ら再評価し即座に修正案を提示します。
- 応答末尾で簡潔に次の要望や調整の希望を促します。
- できないこと (実際の決済・実在在庫照会など) は率直に伝えつつ代替案を添えます。

【目的】
ユーザーの嗜好理解を反映し効率的かつ納得感のある旅行計画を支援し、エラー発生時でもフォールバックにより安定した体験を提供します。
"""
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin()],
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
)

In [25]:
# このセルのポイント:
# - invoke_stream で遅延観察と部分応答更新が可能です。
# - FunctionCallContent をバッファし最終 JSON を再構築、途中断片の誤解析を防ぎます。
# - thread を保持し文脈(過去発話)を引き継いだ連続対話を実現します。
# - current_function_name / argument_buffer で並行呼出を追跡し拡張しやすくします。
from IPython.display import display, HTML
user_inputs = ['バルセロナ行きフライトを調べて']
thread: ChatHistoryAgentThread | None = None

async def main():
    global thread
    for user_input in user_inputs:
        html_output = (f'<div style="margin-bottom:10px"><div style="font-weight:bold">User:</div><div style="margin-left:20px">{user_input}</div></div>')
        full_response, function_calls = [], []
        current_function_name, argument_buffer = None, ''
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            for item in list(response.items):
                if isinstance(item, FunctionCallContent):
                    if item.function_name: current_function_name = item.function_name
                    if isinstance(item.arguments, str): argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    if current_function_name:
                        formatted = argument_buffer.strip()
                        try:
                            parsed = json.loads(formatted)
                            formatted = json.dumps(parsed, ensure_ascii=False)
                        except Exception: pass
                        function_calls.append(f'Calling function: {current_function_name}({formatted})')
                        current_function_name, argument_buffer = None, ''
                    function_calls.append('Function Result: ' + str(item.result))
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)
        if function_calls:
            html_output += ('<div style="margin-bottom:10px"><details><summary style="cursor:pointer; font-weight:bold; color:#0066cc;">Function Calls (click)</summary>' + '<div style="margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;">' + '\n'.join(function_calls) + '</div></details></div>')
        html_output += (f'<div style="margin-bottom:20px"><div style="font-weight:bold">{AGENT_NAME}:</div><div style="margin-left:20px; white-space:pre-wrap">{''.join(full_response)}</div></div><hr>')
        display(HTML(html_output))

await main()